In [1]:
# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

# General libraries.
import re
import numpy as np
import matplotlib.pyplot as plt

# SK-learn libraries for learning.
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.grid_search import GridSearchCV

# SK-learn libraries for evaluation.
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report

# SK-learn libraries for feature extraction from text.
from sklearn.feature_extraction.text import *

In [2]:
speech = open("speech.txt").readlines()
print len(speech)



71211


In [9]:
print speech[:5]

['7, 08, marcus,  abajo  get him down \n', '7, 08, house,  what you got against chickens  one got choked last night thanks to you \n', '7, 08, cuddy,  33yearold male with hemoptysis fever\xe2\x80\xa6 \n', '7, 08, house,  cant we even talk about it \n', '7, 08, cuddy,  you cant apologize we cant talk  and puncture wounds \n']


In [17]:
speech_sample = speech[:5]
print speech_sample
adjusted_speech = [re.split(", ", x)[2:] for x in speech]
print adjusted_speech[:5]
main_chars_speech = [x for x in adjusted_speech if x[0]=="house" or x[0]=="cuddy" or x[0]=="chase" or x[0]=="cameron" \
                     or x[0]=="foreman"]
print len(main_chars_speech)

usable_speech = main_chars_speech

test_labels = [x[0] for x in usable_speech[-10000:-5000]]
test_data = [x[1] for x in usable_speech[-10000:-5000]]
dev_labels = [x[0] for x in usable_speech[-5000:]]
dev_data = [x[1] for x in usable_speech[-5000:]]
train_labels = [x[0] for x in usable_speech[:-10000]]
train_data = [x[1] for x in usable_speech[:-10000]]

print train_labels[:5]
print train_data[:5]


['7, 08, marcus,  abajo  get him down \n', '7, 08, house,  what you got against chickens  one got choked last night thanks to you \n', '7, 08, cuddy,  33yearold male with hemoptysis fever\xe2\x80\xa6 \n', '7, 08, house,  cant we even talk about it \n', '7, 08, cuddy,  you cant apologize we cant talk  and puncture wounds \n']
[['marcus', ' abajo  get him down \n'], ['house', ' what you got against chickens  one got choked last night thanks to you \n'], ['cuddy', ' 33yearold male with hemoptysis fever\xe2\x80\xa6 \n'], ['house', ' cant we even talk about it \n'], ['cuddy', ' you cant apologize we cant talk  and puncture wounds \n']]
40807
['house', 'cuddy', 'house', 'cuddy', 'house']
[' what you got against chickens  one got choked last night thanks to you \n', ' 33yearold male with hemoptysis fever\xe2\x80\xa6 \n', ' cant we even talk about it \n', ' you cant apologize we cant talk  and puncture wounds \n', ' im not apologizing for doing the right thing \n']


In [18]:
print len(train_labels)
print len(train_data)
print len(dev_labels)
print len(dev_data)
print len(test_labels)
print len(test_data)

30807
30807
5000
5000
5000
5000


In [19]:
cv1 = CountVectorizer().fit(train_data)
dtm1 = cv1.transform(train_data)

In [ ]:

lr1 = LogisticRegression().fit(dtm1, train_labels)
preds = lr1.predict(cv1.transform(dev_data))
print metrics.f1_score(dev_labels, preds, average="weighted")

In [22]:
nb1 = MultinomialNB().fit(dtm1, train_labels)
predsNB = nb1.predict(cv1.transform(dev_data))
print nb1.score(cv1.transform(dev_data), dev_labels)
a = [.01,.02,.03,.04,.05,.06,.07,.08,.09,.1,.2,.3,.4,.5,.6,.7,.8,.9,.99,.999,.9999]
nb1_adjusted = GridSearchCV(nb1, {'alpha': a})
nb1_adjusted.fit(dtm1, train_labels)
print nb1_adjusted.score(cv1.transform(dev_data), dev_labels)

0.4814
0.4816


In [ ]:
predsANB = nb1_adjusted.predict(cv1.transform(dev_data))
print